In [1]:
%load_ext autoreload
%autoreload 2

import os
import warnings
import numpy as np
import pandas as pd
import qiskit_metal as metal
from numpy import pi
from scipy.constants import c, h, pi, hbar, e
from qiskit_metal import designs, draw, MetalGUI, Dict, Headings
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength
from qiskit_metal.analyses.quantization import LOManalysis, EPRanalysis
from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem, QuantumSystemRegistry

ws_path = os.getcwd()
warnings.filterwarnings("ignore")

design = designs.DesignPlanar()
design.overwrite_enabled = True

09:30PM 01s INFO [__init__]: TransmonBuilder with system_type TRANSMON registered to QuantumSystemRegistry
09:30PM 01s INFO [__init__]: FluxoniumBuilder with system_type FLUXONIUM registered to QuantumSystemRegistry
09:30PM 01s INFO [__init__]: TLResonatorBuilder with system_type TL_RESONATOR registered to QuantumSystemRegistry
09:30PM 01s INFO [__init__]: LumpedResonatorBuilder with system_type LUMPED_RESONATOR registered to QuantumSystemRegistry


In [2]:
#constants:
phi0    = h/(2*e)
varphi0 = phi0/(2*pi)

# project target parameters
f_qList = np.around(np.linspace(5.25, 5.75, 4),2) # GHz
f_rList = f_qList + 1.8 # GHz
L_JJList = np.around(varphi0**2/((f_qList*1e9+300e6)**2/(8*300e6))/h*1e9, 2) # nH

# initial CPW readout lengths
def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

find_resonator_length(f_rList, 10, 6, 2)

'[8.63311117 8.42983847 8.24707774 8.06138195] mm'

In [3]:
find_resonator_length(np.around(np.linspace(8, 9, 4),2), 10, 6, 2)

'[7.60792921 7.30653465 7.02000388 6.76260374] mm'

In [4]:
chipX = 9
chipY = 9

design.chips.main.size_x = str(chipX)+"mm"
design.chips.main.size_y = str(chipY)+"mm"

gui = MetalGUI(design)

design.delete_all_components()

In [5]:
# basic parameters for inital cut
C_JJ = 2

pad_g = 30
pad_w = 450
pad_h = 150

readout_g  = 30
buslineL_g = 30
buslineH_g = 30

C_JJU = str(C_JJ)+"fF"
pad_g = str(pad_g)+"um"
pad_w = str(pad_w)+"um"
pad_h = str(pad_h)+"um"

readout_g  = str(readout_g)+"um"
buslineL_g = str(buslineL_g)+"um"
buslineH_g = str(buslineH_g)+"um"

connector1_pads_options = dict(
    cR = dict(loc_W=0, loc_H=1, pad_width='80 um', pad_gap=readout_g, pad_height='30 um'),  # readout resonator
    cL = dict(loc_W=1, loc_H=-1, pad_width='80 um', pad_gap=buslineL_g, pad_height='30 um'), # bus line for lower frequency
    cH = dict(loc_W=-1, loc_H=-1, pad_width='80 um', pad_gap=buslineH_g, pad_height='30 um') # bus line for higher frequency
)
connector2_pads_options = dict(
    cR = dict(loc_W=0, loc_H=1, pad_width='80 um', pad_gap=readout_g, pad_height='30 um'),  # readout resonator
    cL = dict(loc_W=1, loc_H=-1, pad_width='80 um', pad_gap=buslineL_g, pad_height='30 um'), # bus line for lower frequency
    cH = dict(loc_W=-1, loc_H=-1, pad_width='80 um', pad_gap=buslineH_g, pad_height='30 um') # bus line for higher frequency
)
connector3_pads_options = dict(
    cR = dict(loc_W=0, loc_H=1, pad_width='80 um', pad_gap=readout_g, pad_height='30 um'),  # readout resonator
    cL = dict(loc_W=1, loc_H=-1, pad_width='80 um', pad_gap=buslineL_g, pad_height='30 um'), # bus line for lower frequency
    cH = dict(loc_W=-1, loc_H=-1, pad_width='80 um', pad_gap=buslineH_g, pad_height='30 um') # bus line for higher frequency
)
connector4_pads_options = dict(
    cR = dict(loc_W=0, loc_H=1, pad_width='80 um', pad_gap=readout_g, pad_height='30 um'),  # readout resonator
    cL = dict(loc_W=1, loc_H=-1, pad_width='80 um', pad_gap=buslineL_g, pad_height='30 um'), # bus line for lower frequency
    cH = dict(loc_W=-1, loc_H=-1, pad_width='80 um', pad_gap=buslineH_g, pad_height='30 um') # bus line for higher frequency
)

In [6]:
# target f: 5.25 
connection1_pads_options = dict(
    B1 = dict(loc_W=1, loc_H=-1),
    R= dict(loc_W=1, loc_H=1),
    B2 = dict(loc_W=-1, loc_H=-1)
)

Q1 = TransmonPocketCL(design, 'Q1',
                      options=dict(pos_x=f'0mm',
                                   pos_y=f'0mm',
                                   pad_gap = pad_g,
                                   pad_width = pad_w,
                                   pad_height = pad_h,
                                   orientation=0,
                                   connection_pads=dict(**connection1_pads_options)))

gui.rebuild()
gui.autoscale()

In [7]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qres = EPRanalysis(design, "hfss")
hfss = eig_qres.sim.renderer
hfss.start()

INFO 09:30PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:30PM [load_ansys_project]: 	Opened Ansys App
INFO 09:30PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 09:30PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/sunpark/Documents/Ansoft/
	Project:   Project48
INFO 09:30PM [connect_design]: No active design found (or error getting active design).
INFO 09:30PM [connect]: 	 Connected to project "Project48". No design detected


True

In [8]:
hfss.activate_ansys_design("Q1Readout", 'eigenmode')

hfss.render_design(['Q1'], [])

# Analysis properties
setup = hfss.pinfo.setup
setup.n_modes = 1
setup.passes = 10
setup.box_plus_buffer =True
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', str(L_JJList[0])+'nH')
pinfo.design.set_variable('Cj', C_JJU)

setup.analyze()

09:30PM 26s WARNING [activate_ansys_design]: The design_name=Q1Readout was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 09:30PM [connect_design]: 	Opened active design
	Design:    Q1Readout [Solution type: Eigenmode]
WARNING 09:30PM [connect_setup]: 	No design setup detected.
WARNING 09:30PM [connect_setup]: 	Creating eigenmode default setup.
INFO 09:30PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:30PM [analyze]: Analyzing setup Setup



Number of eigenmodes to find             = 1
Number of simulation passes              = 10
Convergence freq max delta percent diff  = 0.1



In [9]:
eig_qres.sim.convergence_t, eig_qres.sim.convergence_f, _ = hfss.get_convergences()
eig_qres.sim.plot_convergences()

09:31PM 45s INFO [get_f_convergence]: Saved convergences to C:\Users\sunpark\Documents\GitHub\qiskitHackathon2022\projectB\hfss_eig_f_convergence.csv


In [10]:
import pyEPR as epr

pinfo = hfss.pinfo
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj',
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify

eprd = epr.DistributedAnalysis(pinfo)

print(eprd)

ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

Design "Q1Readout" info:
	# eigenmodes    1
	# variations    1

ℰ_elec_all       = 1.34473606933356e-24
ℰ_elec_substrate = 1.23844721126538e-24
EPR of substrate = 92.1%

ℰ_mag_all       = 8.06931704387712e-27
ℰ_mag % of ℰ_elec_all  = 0.6%



In [11]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)


Variation 0  [1/1]

  Mode 0 at 5.56 GHz   [1/1]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.4%  6.724e-25 4.035e-27

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 96.98%
	jj              0.779194  (+)        0.0242405
		(U_tot_cap-U_tot_ind)/mean=13.21%
Calculating Qdielectric_main for mode 0 (0/0)
p_dielectric_main_0 = 0.9209593164844193


WARNING 09:31PM [__init__]: <p>Error: <class 'IndexError'></p>
ERROR 09:31PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 26.4% is > 15%.                     
Is the simulation converged? Proceed with caution
ERROR 09:31PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 26.4% is > 15%.                     
Is the simulation converged? Proceed with caution



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project48\Q1Readout\2022-02-09 21-31-46.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.305769
dtype: float64

Pm_norm idx =
     jj
0  True
*** P (participation matrix, not normlz.)
         jj
0  0.760753

*** S (sign-bit matrix)
   s_jj
0     1
*** P (participation matrix, normalized.)
      0.99

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       297

*** Chi matrix ND (MHz) 
       335

*** Frequencies O1 PT (MHz)
0    5263.885533
dtype: float64

*** Frequencies ND (MHz)
0    5245.862675
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0]


OrderedDict([('0',
              OrderedDict([('f_0', 0    5561.203589
                            Name: 0, dtype: float64),
                           ('f_1', 0    5263.885533
                            dtype: float64),
                           ('f_ND',
                            0    5245.862675
                            dtype: float64),
                           ('chi_O1',
                                        0
                            0  297.318056),
                           ('chi_ND',
                                        0
                            0  334.534008),
                           ('ZPF', array([[0.46398218]])),
                           ('Pm_normed', array([[0.99336736]])),
                           ('Pm_raw',
                            0    0.760753
                            Name: jj, dtype: float64),
                           ('_Pm_norm',
                            modes
                            0    1.305769
                            d

In [12]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')